<a href="https://colab.research.google.com/github/FARTASH-421/AI2/blob/master/Project_CR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [363]:
# ---------------------------------------------
#
#       Name: Abdul Qadir "Fartash"
#       ID  : 99243100
#       Porject_CR
#
# ---------------------------------------------

#Part A

In [293]:
def mul(num1, num2):
  bin_to_Int = int (num1, 2) *  int(num2, 2)
  result = bin_to_Int % ((2 ** 16) + 1)
  lis = format(result, 'b')
  l = len(lis)
  len_zero = 16 - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front+lis


In [294]:
def sum(num1, num2):
  bin_to_Int = int (num1, 2) +  int(num2, 2)
  result = bin_to_Int % (2 ** 16)
  lis = format(result, 'b')
  l = len(lis)
  len_zero = 16 - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front+lis


In [295]:
def xor(num_1, num_2, size):
  int_x = int(num_1, 2)
  int_y = int(num_2, 2)
  result = int_x ^ int_y
  str_result = format(result, 'b')
  len_zero = size - len(str_result)
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front + str_result

In [296]:
def keyGen(key):

  subKey = []
  for j in range(7):
    sub = [key[i:i+16] for i in range(0, len(key), 16)]
    subKey.extend(sub)
    key = key[25:] + key[:25]

  return subKey

In [297]:
def Round (list_P, list_K):
    P1 = list_P[0 :16]
    P2 = list_P[16:32]
    P3 = list_P[32:48]
    P4 = list_P[48:64]

    step_1 = mul (P1, list_K[0])
    step_2 = sum (P2, list_K[1])
    step_3 = sum (P3, list_K[2])
    step_4 = sum (P4, list_K[3])
    step_5 = xor (step_1, step_3, 16)
    step_6 = xor (step_2, step_4, 16)
    step_7 = mul (step_5, list_K[4])
    step_8 = sum (step_6, step_7)
    step_9 = mul (step_8, list_K[5])
    step_10 = sum (step_7, step_9)

    step_11 = xor (step_1, step_9, 16)
    step_12 = xor (step_3, step_9, 16)
    step_13 = xor (step_2, step_10, 16)
    step_14 = xor (step_4, step_10, 16)
    return step_11 + step_12 + step_13 + step_14



In [298]:
def Block_Cipher(plaintext, key):

  subKey = keyGen(key)
  # print(f'{subKey[:10]}')
  # print(f'-> subkey[0]:{len(subKey[0])}')

  index_key = 0
  for i in range(8):
    part_key = subKey[index_key :(index_key+6)]
    # print(f'size part key: {len(part_key)} -> {part_key}')
    plaintext = Round(plaintext, part_key)
    index_key += 6

  # print(int(plaintext, 2))
  R_1 = plaintext[0:16]
  R_2 = plaintext[16:32]
  R_3 = plaintext[32:48]
  R_4 = plaintext[48:64]


  step_1 = mul (R_1, subKey[index_key])
  step_2 = sum (R_2, subKey[index_key+1])
  step_3 = sum (R_3, subKey[index_key+2])
  step_4 = mul (R_4, subKey[index_key+3])

  ciphertext = step_1 + step_2 +step_3 + step_4
  return ciphertext


#Test Part 1

In [352]:
plaintext = b"\x00\x11\x22\x33\x44\x55\x66\x77"
key = b"\x81\x23\x45\x67\x89\xab\xcd\xef\xfe\xdc\xba\x98\x76\x54\x32\x18"

plainB = ''.join(f'{byte:08b}' for byte in plaintext)
keyB = ''.join(f'{byte:08b}' for byte in key)

print('Hex Plaintext  : ', plaintext)
print('plaintext      : ', plainB)
print('size plaintext : ', len(plainB))

print('Hex Key        : ', key)
print('key            : ', keyB)
print('size key       : ', len(keyB))


print('\n-------------------------------------------------------------------------------------------------------------------------------------------------')
ciphertext = Block_Cipher(plainB, keyB)

print('cipherText      : ', ciphertext)
print('size ciphertext : ', len(ciphertext))

x = int(ciphertext, 2)
hex_bytes = x.to_bytes(8, byteorder='big')
print('Hex cipherText  : ',hex_bytes)


Hex Plaintext  :  b'\x00\x11"3DUfw'
plaintext      :  0000000000010001001000100011001101000100010101010110011001110111
size plaintext :  64
Hex Key        :  b'\x81#Eg\x89\xab\xcd\xef\xfe\xdc\xba\x98vT2\x18'
key            :  10000001001000110100010101100111100010011010101111001101111011111111111011011100101110101001100001110110010101000011001000011000
size key       :  128

-------------------------------------------------------------------------------------------------------------------------------------------------
cipherText      :  0011001000001011100100101110110101001110011001010010000010010000
size ciphertext :  64
Hex cipherText  :  b'2\x0b\x92\xedNe \x90'


#Part B


In [364]:
import random

def CBC(plaintext, key):

  cipher_text_CBC = ''

  # if input is Hex
  bin_plaintext = ''.join(f'{byte:08b}' for byte in plaintext)
  key_bin = ''.join(f'{byte:08b}' for byte in key)

  # if input is char
  # bin_plaintext = ''.join([format(ord(char),'b') for char in plaintext])
  # key_bin = ''.join([format(ord(char),'b') for char in key])

  len_plain = len(bin_plaintext)
  len_key = len(key_bin)


  len_zero = 128 - len_key
  zero_front = ''.join([str(0) for i in range(len_zero)])
  key_bin = zero_front + key_bin

  random_num = int((random.random()*100000) % (2 ** 16))
  bin_rand = format(random_num, 'b')

  for i in range(0,len_plain, 64):

    if((i < len_plain) and ((i+64) > len_plain)):
      index_temp = len_plain - i
      plain_tt = bin_plaintext[i : i+index_temp]

      out_xor = xor(plain_tt, bin_rand, 64)
      bin_rand = Block_Cipher(out_xor, key_bin)
      cipher_text_CBC += bin_rand

    else:

      plain_t = bin_plaintext[i:(i+64)]
      out_xor = xor(plain_t, bin_rand, 64)
      bin_rand = Block_Cipher(out_xor, key_bin)
      cipher_text_CBC += bin_rand

  print(cipher_text_CBC)

  # if input is text then output convert to text
  # results = ''.join([chr((int(cipher_text_CBC[i:i+7], 2)%26) + 65) for i in range(0, len(cipher_text_CBC), 8)])
  # cipher_text = results


  # if input is hex then output convert to hex
  out = int(cipher_text_CBC, 2)
  print(out)
  cipher_text_CBC = out.to_bytes(8, byteorder='big')
  return cipher_text_CBC



#Test Part 2

In [367]:
plaintext = b"\x00\x11\x22\x33\x44\x55\x66\x77"
key = b"\x81\x23\x45\x67\x89\xab\xcd\xef\xfe\xdc\xba\x98\x76\x54\x32\x18"
g = CBC(plaintext, key)
print(g)


0100101010000010111000010110101000010011010100010010110011110010
5369101551485922546
b'J\x82\xe1j\x13Q,\xf2'


#Part C


In [302]:
def UpDataCounterBin(counter):
  counter = counter + 1
  bin_counter = format(counter, 'b')
  zero = ''.join([str(0) for i in range(64 - len(bin_counter))])
  return zero + bin_counter


In [344]:
def CTR(plaintext, key):

  cipher_text_CTR= ''

  # if input is Hex
  plaintext_CTR = ''.join(f'{byte:08b}' for byte in plaintext)
  key_CTR = ''.join(f'{byte:08b}' for byte in key)

  # if input is text
  # plaintext_CTR = ''.join([format(ord(char),'b') for char in plaintext])
  # key_CTR = ''.join([format(ord(char),'b') for char in key])

  len_cipherCTR = len(plaintext_CTR)
  len_keyCTR = len(key_CTR)

  bin_counter = '0'
  len_zero = 128 - len_keyCTR
  zero_front = ''.join([str(0) for i in range(len_zero)])
  bin_key = zero_front + key_CTR


  for i in range(0,len_cipherCTR, 64):

    bin_counter = UpDataCounterBin(int(bin_counter, 2))

    if((i < len_cipherCTR) and ((i+64) > len_cipherCTR)):

      out_EncryptBlock = Block_Cipher(bin_counter, bin_key)
      index_temp = len_cipherCTR - i
      plain_tt = plaintext_CTR[i : i+index_temp]
      out_xor_CTR = xor(plain_tt, out_EncryptBlock, 64)
      cipher_text_CTR += out_xor_CTR

    else:
      out_EncryptBlock = Block_Cipher(bin_counter, bin_key)
      plain_t = plaintext_CTR[i:(i+64)]
      out_xor_CTR = xor(plain_t, out_EncryptBlock, 64)
      cipher_text_CTR += out_xor_CTR

  print(cipher_text_CTR)
  # if input is text then convert ouput to text
  # results = ''.join([chr((int(cipher_text_CTR[i:i+7], 2)%26) + 65) for i in range(0, len(cipher_text_CTR), 8)])
  # cipher_text_CTR = results

  # if input is Hex then convert output to Hex
  out_CTR = int(cipher_text_CTR, 2)
  print(out_CTR)
  cipher_text_CTR = out_CTR.to_bytes(8, byteorder='big')
  return cipher_text_CTR



#Test Part 3


In [361]:
plaintext = b"\x00\x11\x22\x33\x44\x55\x66\x77"
key = b"\x81\x23\x45\x67\x89\xab\xcd\xef\xfe\xdc\xba\x98\x76\x54\x32\x18"
g = CTR(plaintext, key)
print(g)

0010010100110000010010101000101011010010011010110100010011110001
2679723738381632753
b'%0J\x8a\xd2kD\xf1'
